In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


In [2]:
data_dir = f'{main_dir}/Snapshots/1/Test'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/VFeatures')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'label_encoded_data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })
p.save()
p.run()

---------------------------
0: LoaderNode [2019-09-28 12:05:01]
params:
 {'input_directory': '../../../Snapshots/1/Test', 'file': 'label_encoded_data.pkl'}
---------------------------
1: LoaderNode [2019-09-28 12:05:06]
params:
 {'input_directory': '../../../Snapshots/1/Test', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-28 12:05:06]
params:
 {'input_directory': '../../../Snapshots/1/Test', 'file': 'categorical_columns.yaml'}


In [3]:
df = p.data['data'][['isFraud']+[f'V{i}' for i in range(1,340)]]

In [4]:
del p
gc.collect()

40

In [5]:
df

,isFraud,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4170236,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4170237,-1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
########################### Let's find nan groups in V columns
nans_groups = {}
nans_df = df.isna()

i_cols = ['V'+str(i) for i in range(1,340)]
for col in i_cols:
    cur_group = nans_df[col].sum()
    if cur_group>0:
        try:
            nans_groups[cur_group].append(col)
        except:
            nans_groups[cur_group]=[col]
            
for n_group,n_members in nans_groups.items():
    print(n_group, len(n_members))

455805 11
88662 23
245823 18
89995 22
101245 20
314 43
939501 18
939225 11
820866 31
821037 19
840073 46
818499 16
15 32
7300 11
938449 18


In [7]:
test_group = nans_groups[840073]

In [12]:
pca.transform(df[sc_test_group]).shape

(1097231, 10)

In [23]:
MIN_VARIANCE=0.95
df_v = pd.DataFrame(index=df.index)
for test_group_name,test_group in nans_groups.items():
    print ("Test group: ", test_group)
    from sklearn.preprocessing import StandardScaler
    for col in test_group:
        sc = StandardScaler()
        sc.fit(df[df.isFraud>=0][[col]].fillna(0))
        df[col+'_sc'] = sc.transform(df[[col]].fillna(0))

    sc_test_group = [col+'_sc' for col in test_group]  

    # check -> same obviously
    features_check = []

    from sklearn.decomposition import PCA
    pca = PCA(random_state=44)
    pca.fit(df[df.isFraud>=0][sc_test_group])
    print(len(sc_test_group), pca.transform(df[df.isFraud>=0][sc_test_group]).shape[-1])
    trans = pca.transform(df[sc_test_group])

    sc_variance =pca.explained_variance_ratio_
    print(sc_variance)
    for i in range(len(sc_variance)):
        explained = sum(sc_variance[:i])
        if explained >=MIN_VARIANCE:
            print (f"{i} columns -- {explained} explained")
            break
    reduced_cols = [f'V_PCA_{test_group_name}_{d}' for d in range(i)]
    for col in reduced_cols:
        df_v[col] = 0
    df_v[reduced_cols] = trans[:,:i]
    
#     break


Test group:  ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11']
11 11
[7.98182617e-01 1.25703590e-01 4.56086055e-02 8.82022050e-03
 8.18611482e-03 4.44338226e-03 3.71838278e-03 1.93109869e-03
 1.63918636e-03 1.18137291e-03 5.85428353e-04]
3 columns -- 0.969494813324809 explained
Test group:  ['V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34']
23 23
[4.14213039e-01 2.49744058e-01 8.67945627e-02 7.26703354e-02
 4.76893993e-02 4.41823220e-02 2.83346247e-02 1.85122328e-02
 7.67237729e-03 6.61576297e-03 4.78963632e-03 3.80839404e-03
 3.23812050e-03 2.52224975e-03 1.92569545e-03 1.81557749e-03
 1.52346419e-03 1.16852787e-03 1.10150266e-03 9.98878129e-04
 4.81334627e-04 1.52890574e-04 4.50142448e-05]
8 columns -- 0.9621405739079144 explained
Test group:  ['V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', '

18 18
[6.45587873e-01 1.25760516e-01 9.77851006e-02 6.87303484e-02
 2.23054195e-02 1.93903041e-02 1.09825395e-02 3.02665596e-03
 2.37627339e-03 1.35121823e-03 1.00661867e-03 5.15340137e-04
 4.63574882e-04 3.31437875e-04 2.39500275e-04 1.40335302e-04
 4.89283400e-06 2.05169400e-06]
5 columns -- 0.9601692571182745 explained


In [24]:
df_v.to_pickle(f'{main_dir}/Snapshots/1/VFeatures/v_features_reduced.pkl')

In [25]:
df_v.head()

,V_PCA_455805_0,V_PCA_455805_1,V_PCA_455805_2,V_PCA_88662_0,V_PCA_88662_1,V_PCA_88662_2,V_PCA_88662_3,V_PCA_88662_4,V_PCA_88662_5,V_PCA_88662_6,...,V_PCA_7300_2,V_PCA_7300_3,V_PCA_7300_4,V_PCA_7300_5,V_PCA_7300_6,V_PCA_938449_0,V_PCA_938449_1,V_PCA_938449_2,V_PCA_938449_3,V_PCA_938449_4
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,2.248276,-1.498574,0.173556,-0.902137,1.259157,-0.091400,-1.580449,0.036157,-0.282162,-0.185864,...,-0.179607,-0.107766,0.304612,0.281329,-0.141240,-0.217517,-0.029469,-0.011814,0.065417,0.005769
2987001,-3.006330,0.080073,0.094644,-0.497435,0.286989,-0.062312,-0.575823,-1.836372,0.938325,0.063590,...,-0.179607,-0.107766,0.304612,0.281329,-0.141240,-0.217517,-0.029469,-0.011814,0.065417,0.005769
2987002,2.248276,-1.498574,0.173556,-0.902137,1.259157,-0.091400,-1.580449,0.036157,-0.282162,-0.185864,...,-0.179607,-0.107766,0.304612,0.281329,-0.141240,-0.217517,-0.029469,-0.011814,0.065417,0.005769
2987003,-3.006330,0.080073,0.094644,-0.902137,1.259157,-0.091400,-1.580449,0.036157,-0.282162,-0.185864,...,0.289877,-0.091256,-0.048588,-0.327969,0.268265,-0.217517,-0.029469,-0.011814,0.065417,0.005769
2987004,-3.006330,0.080073,0.094644,-1.981801,-5.438388,-0.134723,-0.002586,-0.010092,-0.596549,-0.108221,...,-0.179607,-0.107766,0.304612,0.281329,-0.141240,-0.217517,-0.029469,-0.011814,0.065417,0.005769
